In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\bhagy\Downloads\archive (2)\Training Dataset.csv")
df = df.dropna()
docs = df.apply(lambda row:
    f"Applicant ID {row['Loan_ID']} is a {row['Gender']} with {row['Education']} education. "
    f"Married: {row['Married']}, Dependents: {row['Dependents']}, Self-Employed: {row['Self_Employed']}. "
    f"ApplicantIncome: {row['ApplicantIncome']}, CoapplicantIncome: {row['CoapplicantIncome']}, "
    f"LoanAmount: {row['LoanAmount']}, Loan_Term: {row['Loan_Amount_Term']}, "
    f"Credit_History: {row['Credit_History']}, Property_Area: {row['Property_Area']}. "
    f"Loan_Status: {'Approved' if row['Loan_Status'] == 'Y' else 'Rejected'}.",
axis=1).tolist()

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
doc_embeddings = vectorizer.fit_transform(docs)


In [3]:
query = "What are the factors affecting loan approval?"
query_embedding = vectorizer.transform([query])

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(query_embedding, doc_embeddings).flatten()
top_k_indices = similarities.argsort()[-10:][::-1]
retrieved_docs = [docs[i] for i in top_k_indices]

In [5]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

context = " ".join(retrieved_docs)
print("\n\n".join(retrieved_docs))
result = qa_model(question=query, context=context)
print(result['answer'])


Device set to use cpu


Applicant ID LP002990 is a Female with Graduate education. Married: No, Dependents: 0, Self-Employed: Yes. ApplicantIncome: 4583, CoapplicantIncome: 0.0, LoanAmount: 133.0, Loan_Term: 360.0, Credit_History: 0.0, Property_Area: Semiurban. Loan_Status: Rejected.

Applicant ID LP002984 is a Male with Graduate education. Married: Yes, Dependents: 2, Self-Employed: No. ApplicantIncome: 7583, CoapplicantIncome: 0.0, LoanAmount: 187.0, Loan_Term: 360.0, Credit_History: 1.0, Property_Area: Urban. Loan_Status: Approved.

Applicant ID LP001664 is a Male with Graduate education. Married: No, Dependents: 0, Self-Employed: No. ApplicantIncome: 4191, CoapplicantIncome: 0.0, LoanAmount: 120.0, Loan_Term: 360.0, Credit_History: 1.0, Property_Area: Rural. Loan_Status: Approved.

Applicant ID LP001665 is a Male with Graduate education. Married: Yes, Dependents: 1, Self-Employed: No. ApplicantIncome: 3125, CoapplicantIncome: 2583.0, LoanAmount: 170.0, Loan_Term: 360.0, Credit_History: 1.0, Property_Area: